In [2]:
import numpy as np
import librosa
import librosa.display
import os
from matplotlib import pyplot as plt
import random

In [3]:
# All mfcc data are contained in numpy_datas, which is a list. 
# The each entry of the list is another list consistes of [mfccs, file name]
MFCCs_DATA = "MFCCsData"
numpy_datas = []

dirlist = os.listdir(MFCCs_DATA)
for d in dirlist:
    d = os.path.join(MFCCs_DATA, d)
    datalist = os.listdir(d)
    datalist = [[np.load(os.path.join(d,x)), os.path.join(d,x)] for x in datalist]
    numpy_datas.extend(datalist)

In [4]:
numpy_datas[200][0].shape

(19, 612)

In [5]:
numpy_datas[100][0].shape

(19, 492)

Notice that the size of mfccs for each emotion file is different, need to resize the data into the same size

In [6]:
for i in range(len(numpy_datas)):
    numpy_datas[i][0] = np.resize(numpy_datas[i][0], (19,512))

In [7]:
numpy_datas[200][0].shape

(19, 512)

In [8]:
numpy_datas[100][0].shape

(19, 512)

So now all data has the same shape

Separate the data list according to the label. Change file name to correct label. [1,0,0] is angry, [0,1,0] is happy, [0,0,1] is normal.

In [9]:
# change file name to correct label
angry = []
happy = []
normal = []

for i in range(len(numpy_datas)):
    file_name = numpy_datas[i][1]
    if "angry" in file_name:
        numpy_datas[i][1] = np.array([1,0,0])
        angry.append(numpy_datas[i])
    elif "happy" in file_name:
        numpy_datas[i][1] = np.array([0,1,0])
        happy.append(numpy_datas[i])
    else:
        numpy_datas[i][1] = np.array([0,0,1])
        normal.append(numpy_datas[i])

Now split the Train-Valid-Test data with a ratio of 60%-20%-20%

In [10]:
# first shuffle the list of each emotion
random.shuffle(angry)
random.shuffle(happy)
random.shuffle(normal)

In [11]:
# then select the first 180 in each emotion as train data, first 60 in each emotion as valid data, and first 60 in each emotion as test data
train_data = angry[:int(len(angry)*0.6)] + happy[:int(len(happy)*0.6)] + normal[:int(len(normal)*0.6)]
valid_data = angry[int(len(angry)*0.6):int(len(angry)*(0.6+0.2))] + happy[int(len(happy)*0.6):int(len(happy)*(0.6+0.2))] + normal[int(len(normal)*0.6):int(len(normal)*(0.6+0.2))]
test_data = angry[int(len(angry)*(0.6+0.2)):] + happy[int(len(happy)*(0.6+0.2)):] + normal[int(len(normal)*(0.6+0.2)):]
random.shuffle(train_data)
random.shuffle(valid_data)
random.shuffle(test_data)

In [12]:
train_data

[[array([[  77.281334 ,   72.44777  ,   57.928265 , ...,  138.46736  ,
           138.85919  ,  141.28796  ],
         [ 143.2983   ,  138.08954  ,  130.62244  , ...,  -42.450966 ,
           -60.46024  ,  -76.7444   ],
         [ -85.312805 ,  -90.857315 , -101.586945 , ...,  -37.856365 ,
           -34.997498 ,  -32.629433 ],
         ...,
         [   3.5411825,    2.1870093,    0.6932521, ...,  -10.311531 ,
           -13.193217 ,  -16.86633  ],
         [ -14.289646 ,  -14.970854 ,  -18.770247 , ...,  -35.938213 ,
           -40.46882  ,  -42.0286   ],
         [ -44.042    ,  -46.67802  ,  -41.74465  , ...,  -39.196445 ,
           -37.032425 ,  -44.15297  ]], dtype=float32),
  array([0, 0, 1])],
 [array([[ 96.88104   , 161.35248   , 191.4223    , ..., 122.0983    ,
          135.53354   , 137.63422   ],
         [141.34828   , 150.69981   , 145.39099   , ..., -25.974203  ,
           -9.00708   ,  -2.8903546 ],
         [-19.54163   , -41.133957  , -45.09986   , ...,   7.2737427

In [13]:
# train, valid, test data are all in list consistes of [mfccs, file name]
# want to change the list to two outputs:
# X: vector of mfccs
# Y: vector of labels
def data2vector(data):
    X = []
    Y = []
    for i in range(len(data)):
        X.append(data[i][0])
        Y.append(data[i][1])
    X = np.array(X)
    Y = np.array(Y)
    return X,Y

In [14]:
X_train, Y_train = data2vector(train_data)

In [37]:
print(X_train.shape)
print(Y_train.shape)

(540, 19, 512)
(540, 3)


In [17]:
X_valid, Y_valid = data2vector(valid_data)
X_test, Y_test = data2vector(test_data)

Now let's do normalization, by the value of train data

In [30]:
X_train_miu = np.average(X_train, axis=0)
X_train_std = np.std(X_train, axis=0)

In [33]:
print(X_train_miu.shape)
print(X_train_std.shape)

(19, 512)
(19, 512)


In [38]:
X_train_norm = (X_train - X_train_miu)/X_train_std
X_valid_norm = (X_valid - X_train_miu)/X_train_std
X_test_norm = (X_test - X_train_miu)/X_train_std